# Using TPUs with Vertex AI Training - Hello World

In [4]:
import os
import sys
import shutil
import time

from pathlib import Path

import tensorflow as tf

from google.cloud import aiplatform as aip

In [5]:
print(tf.__version__)
print(aip.__version__)

2.7.0
1.11.0


## Configure environment

In [6]:
PROJECT_ID = 'jk-mlops-dev'
REGION = 'us-central1'
STAGING_BUCKET = 'gs://jk-tpu-staging'
VERTEX_SA = f'vertex-sa@{PROJECT_ID}.iam.gserviceaccount.com'

IMAGE_NAME = 'tpu-test'
IMAGE_URI = f'gcr.io/{PROJECT_ID}/{IMAGE_NAME}'

# Use TPU Accelerators. Temporarily using numeric codes, until types are added to the SDK
#   6 = TPU_V2
#   7 = TPU_V3
TRAIN_TPU, TRAIN_NTPU = (7, 8)
TRAIN_COMPUTE = "cloud-tpu"

## Build a custom training container

In [7]:
TRAINER_FOLDER = 'trainer'

path = Path(TRAINER_FOLDER)
if path.exists():
    shutil.rmtree(path)
path.mkdir() 

In [8]:
%%writefile {TRAINER_FOLDER}/__init__.py
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


Writing trainer/__init__.py


In [23]:
%%writefile {TRAINER_FOLDER}/task.py
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import tensorflow as tf

from typing import Any, Mapping, MutableMapping, Optional, Sequence, Union

from absl import logging
from absl import flags
from absl import app


def train_eval():
    print("In train_eval")
    cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")
    tf.config.experimental_connect_to_cluster(cluster_resolver)
    tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))

def _main(argv):
    train_eval()


FLAGS = flags.FLAGS

flags.DEFINE_list('training_data_paths', None, 'Paths to training datasets')
flags.DEFINE_list('validation_data_paths', None, 'Paths to validation datasets') 
flags.DEFINE_integer('tpu_cores', 8, 'A number of TPU cores')
flags.DEFINE_integer('tpu_type', 6, 'TPU type: 6 = TPU_V2, 7 = TPU_V3')


if __name__=='__main__':
    #flags.mark_flags_as_required([
    #    'fasta_path',
    #    'database_paths',
    #    'output_dir'
    #])
    app.run(_main)



Overwriting trainer/task.py


In [24]:
%%writefile Dockerfile

FROM python:3.8

WORKDIR /
ADD trainer /trainer

RUN wget https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/libtpu/20210525/libtpu.so -O /lib/libtpu.so
RUN chmod 777 /lib/libtpu.so

RUN wget https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/tensorflow/20210525/tf_nightly-2.6.0-cp38-cp38-linux_x86_64.whl
RUN pip3 install tf_nightly-2.6.0-cp38-cp38-linux_x86_64.whl
RUN rm tf_nightly-2.6.0-cp38-cp38-linux_x86_64.whl


ENTRYPOINT ["python3", "-m", "trainer.task"]

Overwriting Dockerfile


In [25]:
!docker build --quiet -t {IMAGE_URI} .
!docker push {IMAGE_URI}

sha256:98162fda1ccb5b181e98c55b3f824af28d3446243fde0061f70ee0cbfba0b1c9
Using default tag: latest
The push refers to repository [gcr.io/jk-mlops-dev/tpu-test]

c4a281fa: Preparing 
8a3d6881: Preparing 
158cd9e6: Preparing 
f6557a94: Preparing 
66d296a6: Preparing 
0d73f0d2: Preparing 
75d6a9af: Preparing 
c52d3702: Preparing 
f209746d: Preparing 
d87a050a: Preparing 
178e6d86: Preparing 
42081bbe: Preparing 
2f416458: Preparing 
be289189: Preparing 
a3d6881: Pushed   1.151GB/1.138GBlatest: digest: sha256:403b181b6d20b8a8f115c0c989014e0a582a629b424cb897e8845450fffdbcfd size: 3484


## Configure and submit a custom training job

### Initialize Vertex SDK

In [12]:
aip.init(project=PROJECT_ID, staging_bucket=STAGING_BUCKET)

### Configure worker pool specs

In [13]:
worker_pool_specs = [
    {
        "container_spec": {
            #"args": TRAINER_ARGS,
            "image_uri": IMAGE_URI,
        },
        "replica_count": 1,
        "machine_spec": {
            "machine_type": TRAIN_COMPUTE,
            "accelerator_type": TRAIN_TPU,
            "accelerator_count": TRAIN_NTPU,
        },
    }
]

print(worker_pool_specs[0])

{'container_spec': {'image_uri': 'gcr.io/jk-mlops-dev/tpu-test'}, 'replica_count': 1, 'machine_spec': {'machine_type': 'cloud-tpu', 'accelerator_type': 7, 'accelerator_count': 8}}


### Submit a job

In [14]:
display_name = f'tpu_hello_world_{time.strftime("%Y%m%d_%H%M%S")}'

job = aip.CustomJob(display_name=display_name, worker_pool_specs=worker_pool_specs)

In [15]:
job.run(sync=True)

INFO:google.cloud.aiplatform.jobs:Creating CustomJob
INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/895222332033/locations/us-central1/customJobs/1778434219404427264
INFO:google.cloud.aiplatform.jobs:To use this CustomJob in another session:
INFO:google.cloud.aiplatform.jobs:custom_job = aiplatform.CustomJob.get('projects/895222332033/locations/us-central1/customJobs/1778434219404427264')
INFO:google.cloud.aiplatform.jobs:View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/1778434219404427264?project=895222332033
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/1778434219404427264 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/1778434219404427264 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/

RuntimeError: Job failed with:
code: 1
message: "CANCELED"
